# Modelado de tópicos con BERTopic


Para compreder acerca de qué temas se habla en un conjunto de documentos, es necesario realizar un análisis de tópicos. El análisis de tópicos es una técnica de minería de texto que permite identificar los temas principales que se tratan en un conjunto de documentos. Exiten una gran cantidad de algoritmos que permiten realizar este análisis, entre ellos destacan *Latent Dirichlet Allocation, Latent Semantic Analysis, Non-negative Matrix Factorization*, entre otros.

Siendo la mayoria de estos métodos basados en la representación de los documentos como vectores de palabras, es decir, se representan los documentos como vectores de frecuencias de palabras, lo que implica que se pierde la información de contexto de las palabras.

En el año 2022 se lanzó [BERTopic](https://arxiv.org/abs/2203.05794), un algoritmo de análisis de tópicos basado en la representación de los documentos como vectores de palabras, pero utilizando la representación de palabras de cualquier modelo transformer de lenguaje (como BERT), a modo que la información contextual contenida dentro de cada texto sea aprovechada. BERTopic es un algoritmo de análisis de tópicos auto-,semi- y supervisado, lo que significa que no es necesario etiquetar todos los documentos para realizar el análisis de tópicos.

El modelo de BERTopic permite agrupar textos cuya semántica es similar y provee de una jerarquía de palabras simples o compuestas para cada grupo de textos. Esta jerarquía de palabras permite al usuario asociar un tema o conjunto de temas a cada grupo.
El funcionamiento de este modelo o algoritmo se divide en tres etapas:

* **Generación y extracción de embbedings por documento**: En esta etapa se utiliza un modelo transformer de lenguaje para generar los embeddings de cada documento. Los embeddings son vectores de números reales que representan la información contextual de cada documento.

* **Agrupamiento de los vectores de documentos en grupos semánticamente similares**: En esta etapa se utiliza un algoritmo de clustering para agrupar los vectores de documentos en grupos semánticamente similares. El algoritmo de clustering utilizado es DBSCAN, el cual permite identificar grupos de documentos que se encuentran cercanos entre sí y que no se encuentran cercanos a los grupos de otros documentos.

* **Creación de la representación de palabras o tópicos para cada grupo**: Para asignar a cada grupo un conjunto de palabras claves, los textos asignados a cada grupo son considerados como un único documento, para el cual se mide la importancia de las palabras contenidas a partir de la frecuencia de aparición. Esto se realiza gracias a la medida estadística conocida como *Term frequency – Inverse document frequency* (Tf-Idf). De esta manera, a partir de las palabras más importantes de cada grupo se obtiene una descripción del tópico.

En este notebook se presenta un ejemplo de aplicación de BERTopic semisupervisado para el análisis de tópicos de un conjunto de documentos. Se presentarán algunas gráficas que permiten visualizar algunos hitos dentro de los datos usados.

In [1]:
#Imports

from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')
nltk.download('punkt')
from umap import UMAP

[nltk_data] Downloading package stopwords to /home/pedri0/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/pedri0/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Carga de datos 20 newsgroups

In [2]:
data = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))
documents = data['data']
label_categories = data['target']
categories = data['target_names']
categories

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [3]:
#dataframe creation
wanted_keys = ['data', 'target']
docs = [data.get(key) for key in wanted_keys]
news = pd.DataFrame(docs).T
news.columns = ['data', 'target']
news['target_name'] = news['target'].apply(lambda x: categories[x])
news['synthetic_target'] = news['target']
news = news.sample(frac=0.25).reset_index(drop=True)
news.sample(5)

,data,target,target_name,synthetic_target
2490,\n: >There is an emergency oxygen system that ...,14,sci.space,14
3688,\n,6,misc.forsale,6
2667,"On Tuesday, when it was raining in Chicago, ES...",10,rec.sport.hockey,10
4388,"Well, here it is, NHL in the year 2000.\nI got...",10,rec.sport.hockey,10
2329,I've been following discussions about the Delt...,14,sci.space,14


In [4]:
#remove some labels
labels = news.target.unique()
for label in labels:
    index_label = news[news.target==label].sample(frac=0.7).index
    news.loc[index_label, 'synthetic_target'] = -1

In [5]:
news.synthetic_target.value_counts()

-1     3297
 13      80
 5       80
 10      79
 11      78
 12      77
 3       77
 4       75
 16      75
 6       73
 9       73
 8       72
 7       72
 14      71
 17      69
 2       68
 1       68
 15      67
 0       58
 18      55
 19      48
Name: synthetic_target, dtype: int64

In [6]:
#add our stop word through countvectorizer model, set to 2 the limit of n-grams
# with min_df: ignore terms that have a document frequency strictly lower than the given threshold (5)
vectorizer_model = CountVectorizer(stop_words=stopwords.words('english'), ngram_range=(1,2), min_df=1)

#use paraphrase-multilingual-mpnet-base-v2 as embedding model. This model is the most accurate for multilingual
#diversity is a parameter in BERTopic to diversity words in each topic such that we limit the number of 
#duplicate words we find in each topic. This is done using an algorithm called Maximal Marginal Relevance 
#which compares word embeddings with the topic embedding.
#We do this by specifying a value between 0 and 1, with 0 being not at all diverse and 1 being completely diverse
#min_topic_size is used to specify what the minimum size of a topic can be. 
#The lower this value the more topics are created.
#calculate_probabilities lets you calculate the probabilities of each topic to each document.
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2", verbose=True,
                      vectorizer_model=vectorizer_model, diversity=0.3, min_topic_size=5,
                      calculate_probabilities=False)

In [7]:
#fit the model using semisupervised topic modeling and infer clusters, probabilities
topics = topic_model.fit_transform(news.data.to_list(), y=news.synthetic_target.tolist())

Batches:   0%|          | 0/148 [00:00<?, ?it/s]

2022-10-12 14:08:42,293 - BERTopic - Transformed documents to Embeddings
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2022-10-12 14:09:02,358 - BERTopic - Reduced dimensionality
2022-10-12 14:09:02,576 - BERTopic - Clustered reduced embeddings


In [10]:
#save inferred clusters of topics
news['cluster_topic'] = topics[0]

In [12]:
news.cluster_topic.value_counts()

-1      1387
 0       454
 1       250
 2       171
 3       115
        ... 
 111       6
 113       5
 115       5
 112       5
 114       5
Name: cluster_topic, Length: 117, dtype: int64

In [14]:
#get dataframe with info about clusters from fitted model
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1387,-1_ax max_file_edu_a86
1,0,454,0_games_season_det_03
2,1,250,1_patients_health_cancer_disease
3,2,171,2_encryption_nsa_law enforcement_clipper chip
4,3,115,3_fbi_agents_warrant_would
...,...,...,...
112,111,6,111_cpu fans_cpu fan_fan blowing_sink grease
113,112,5,112_messages posted_sarcastic sort_funny thing...
114,113,5,113_companies_columbus_addresses_library center
115,114,5,114_mileage_odometer_oxygen sensor_eeprom


In [15]:
#get top 10 words per topic
topic_model.get_topics()

{-1: [('ax max', 0.005801951251383066),
  ('file', 0.0037244025924846194),
  ('edu', 0.00309976386047196),
  ('a86', 0.002656104732012123),
  ('information', 0.0025747185335116864),
  ('system', 0.0023039551733029353),
  ('145 145', 0.0021230078141801414),
  ('pl', 0.002094804466525847),
  ('a86 a86', 0.002039342860958741),
  ('program', 0.001989941473852277)],
 0: [('games', 0.006880379813224154),
  ('season', 0.005942210107381999),
  ('det', 0.005287226769148705),
  ('03', 0.005237657890152873),
  ('hockey', 0.00498700452731302),
  ('baseball', 0.004742235704740176),
  ('players', 0.004626523805434089),
  ('shots', 0.004576946582266683),
  ('stl', 0.004400876263038574),
  ('power play', 0.00413545501699192)],
 1: [('patients', 0.007442527410495439),
  ('health', 0.006279988972118991),
  ('cancer', 0.0057398355293025016),
  ('disease', 0.005599757043385022),
  ('yeast', 0.004894638326597243),
  ('diet', 0.004765705351829008),
  ('effects', 0.004394723231868954),
  ('msg', 0.0042421990

In [16]:
#save model
topic_model.save("bertopic_20news.model")

/opt/conda/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [16]:
topic_model.visualize_barchart()

In [17]:
topic_model.visualize_hierarchy()

### Búsqueda de tópicos por palabra clave

In [24]:
similar_topics, similarity = topic_model.find_topics("GPU and CPU", top_n=5)
topic_model.get_topic(similar_topics[1])

[('vga', 0.030664013491536714),
 ('colors', 0.028678484277278943),
 ('mode 13h', 0.02249771338903429),
 ('vga mode', 0.020473770702294145),
 ('turbo debugger', 0.017503457813295892),
 ('video memory', 0.01564769810598645),
 ('screen', 0.013968407795701583),
 ('256', 0.013560353710016052),
 ('video card', 0.013044831650927086),
 ('svga', 0.013044831650927086)]

In [26]:
news[news.cluster_topic==similar_topics[1]].sample(5)

,data,target,target_name,synthetic_target,cluster_topic
2951,"Hi there,\n\nI'm looking for help on hi-rez CG...",1,comp.graphics,1,78
4681,Experiences with Diamond Viper VLB video card\...,3,comp.sys.ibm.pc.hardware,-1,78
1294,Has anybody gotten CVIEW to work in 32k or 64k...,1,comp.graphics,-1,78
2267,"Hi there,\n\nI've made a VGA mode 13h graphics...",1,comp.graphics,1,78
907,"I have an ATI Graph. Ultra Pro VLB w/2 megs, a...",1,comp.graphics,-1,78
